In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from commpy.filters import rcosfilter
from scipy import signal
from scipy.fft import fft, fftfreq

from ipywidgets import interact
import ipywidgets as widgets

In [2]:
# Signal setting & Amplitude Modulation

fc = 20*10**3   #carrier signal frequency of 20kHz

fs = 2**(math.ceil(np.log2(fc))+2) # sampling frequency, power of 2 that is higer than 2*fc
t = np.linspace(0,1,fs)  
f = fftfreq(fs, 1/fs)

# Carrier signal
Ac = 5 # Amplitude
xc = Ac * np.cos(2 * np.pi * fc * t) # carrier signal
xc_f = abs(fft(xc))/xc.size #Fourier transform / freq domain

# Message signal
fm = 500 # modulation signal frequency 500 Hz
m = 0.5 # modulation index 50% (Normalized so that the amplitude doesn't exceed 1)
xm = m*np.cos(2* np.pi * fm * t) 
xm_f = abs(fft(xm))/xm.size 

# Modulated signal
y = (1+xm)*xc
y_f = abs(fft(y))/y.size


fig = make_subplots(rows = 3, cols = 2, specs=[[{"rowspan": 3},{}],[None, {}],[None, {}]],subplot_titles=("time", "freq"))
fig.update_layout(title="Amplitude Modulation")
fig.add_scatter(x = t, y = xm, name = "message signal", mode = 'lines',row = 1, col = 1)
fig.add_scatter(x = t, y = xc, name = "carrier signal", mode = 'lines',row = 1, col = 1)
fig.add_scatter(x = t, y = y, name = "modulated signal", mode = 'lines',row = 1, col = 1)

fig.add_scatter(x = f, y = xc_f, name = "carrier signal", row = 1, col = 2)
fig.add_scatter(x = f, y = xm_f,name = "message signal", row = 2, col = 2)
fig.add_scatter(x = f, y = y_f, name = "modulated signal", row = 3, col = 2)

fig.update_xaxes(range=[0, 0.005], row =1, col = 1)
fig.update_xaxes(range=[-40000, 40000], row =1, col = 2)
fig.update_xaxes(range=[-40000, 40000], row =2, col = 2)
fig.update_xaxes(range=[-40000, 40000], row =3, col = 2)
figw = go.FigureWidget(fig)
figw



FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'message signal',
              'type': 's…

In [3]:
# Butterworth Lowpass Filter

fc_filt = 1000 # cutoff frequency of filter

fig = make_subplots(rows = 1, cols = 2, subplot_titles=('mag', 'phase'))

butter = [] #butterworth filter coefficients

fig.update_xaxes(type="log", row = 1, col = 1)
fig.update_xaxes(type="log",row=1, col = 2)
fig.update_layout(title="Butterworth lowpass filter")

"""
In the function signal.butter(N, Wn, ... ,fs), Wn is the critical frequency. 
For digital filters, Wn are in the same units as fs. By default, fs is 2 half-cycles/sample, so these are normalized from 0 to 1, where 1 is the Nyquist frequency. (Wn is thus in half-cycles / sample.)
"""


for Nfilt in range(1, 4, 1):
    b, a = signal.butter(Nfilt, fc_filt/(fs/2), 'lowpass', False, 'ba') #divided by fs/2 (nyquist) to match the unit
    butter.append([b, a])
    w, h = signal.freqz(b,a,fs = fs)
    fig.add_scatter(x = w, y = 20*np.log10(abs(h)), name = "butterworth filter of oder" + str(Nfilt)  ,row = 1, col = 1)
    fig.add_scatter(x = w, y = np.rad2deg(np.unwrap(np.angle(h))), name = "butterworth filter phase of oder" + str(Nfilt) , row = 1, col = 2)



fig.update_yaxes(range=[-60,10], row =1, col = 1)
fig.update_xaxes(range=[0, 4], row =1, col = 1)

fig.update_xaxes(range=[0, 4.5], row =1, col = 2)
fig.update_yaxes(range=[-300, 0], row =1, col = 2)

figw = go.FigureWidget(fig)

figw

FigureWidget({
    'data': [{'name': 'butterworth filter of oder1',
              'type': 'scatter',
         …

In [4]:
# Demodulation

yr = abs(y) # rectified signal

fig = go.FigureWidget()
fig.update_layout(title="Demodulation")
fig.add_scatter(x = t, y = xm, name = "message signal", mode = 'lines')

yfilt = []
for i in range(3):
    filteredsignal = signal.lfilter(butter[i][0],butter[i][1], yr) #demodulation
    yfilt.append(filteredsignal)
    fig.add_scatter(x = t, y = yfilt[i], name = "filtered with LPfilter oder" + str(i+1), mode = 'lines')


fig.update_xaxes(range=[0, 0.005])
fig


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'message signal',
              'type': 's…

Higer oder filter gives better reconstruction of the original signal. <br>
It has less noise <br>
However, creates a delay